In [1]:
from qiskit import *
import numpy as np
from qiskit.visualization import plot_histogram
import qiskit.tools.jupyter
import matplotlib.pyplot as plt
from math import gcd
from numpy.random import randint
from fractions import Fraction

## The Problem: Period Finding

Let’s look at the periodic function:

$$ f(x) = a^x \bmod{N}$$

$$a^r \bmod N = 1 $$ 

a and N are co-prime

## The Solution

Shor’s solution was to use quantum phase estimation on the unitary operator to find the period:

$$ U|y\rangle \equiv |ay \bmod N \rangle $$


$$\begin{aligned}
U|1\rangle &= |a_1\rangle & \\
U^2|1\rangle &= |a_2\rangle \\
U^3|1\rangle &= |a_3\rangle \\
& \vdots \\
U^{(r-1)}|1\rangle &= |a_{r-1}\rangle \\
U^r|1\rangle &= |a_1\rangle 
\end{aligned}$$

So a superposition of the states in this cycle ($|u_0\rangle$) would be an eigenstate of $U$:

$$|u_0\rangle = \tfrac{1}{\sqrt{r}}\sum_{k=0}^{r-1}{|a^k \bmod N\rangle} $$

$$\begin{aligned}
|u_1\rangle &= \tfrac{1}{\sqrt{r}}\sum_{k=0}^{r-1}{e^{-\tfrac{2\pi i k}{r}}|a^k \bmod N\rangle}\\[10pt]
U|u_1\rangle &= e^{\tfrac{2\pi i}{r}}|u_1\rangle 
\end{aligned}
$$

$$\begin{aligned}
|u_s\rangle &= \tfrac{1}{\sqrt{r}}\sum_{k=0}^{r-1}{e^{-\tfrac{2\pi i s k}{r}}|a^k \bmod N\rangle}\\[10pt]
U|u_s\rangle &= e^{\tfrac{2\pi i s}{r}}|u_s\rangle 
\end{aligned}
$$

We now have a unique eigenstate for each integer value of $s$ where $$0 \leq s \leq r-1.$$ If we sum up all these eigenstates, the different phases cancel out all computational basis states except $|1\rangle$:

$$ \tfrac{1}{\sqrt{r}}\sum_{s=0}^{r-1} |u_s\rangle = |1\rangle$$



Since the computational basis state $|1\rangle$ is a superposition of these eigenstates, which means if we do QPE on $U$ using the state $|1\rangle$, we will measure a phase:

$$\phi = \frac{s}{r}$$


If we have $r$, we might be able to use this to find a factor of $N$. Since:

$$a^r \bmod N = 1 $$

then:

$$(a^r - 1) \bmod N = 0 $$

and:

$$a^r -1 = (a^{r/2}-1)(a^{r/2}+1)$$

We can find primes of N by computing with random guessed a $$gcd(N,a^{r/2}\pm1)$$ 

## The Quantum Circuit
for a = 7 N = 15

In [9]:
class Shor:
    def __init__(self):
        self.N = 15
        self.m = int(np.ceil(np.log2(self.N))) # Quantum registers
        self.t = 3                             # Classic registers
        self.qc = QuantumCircuit(self.t+self.m, self.t)
        
        self.qc.h(range(self.t)) # Hadamard-gate 
        self.qc.x(self.t) # X-gate inverse
        for i in range(self.t-1):
            self.qc.append(self.multi_7mod15_cgate(i), [i]+ list(range(self.t,self.t+self.m)))

        qft_dag = Shor.qft_qc(self.t).inverse()
        qft_dag.name = 'QFT+'

        self.qc.append(qft_dag, range(self.t))
        self.qc.measure(range(self.t), range(self.t))
    
    def _7mod15_gate(self):
        U_qc = QuantumCircuit(self.m)
        U_qc.x(range(self.m))
        U_qc.swap(1, 2)
        U_qc.swap(2, 3)
        U_qc.swap(0, 3)
        U = U_qc.to_gate()
        U.name ='{}Mod{}'.format(7, self.N)
        return U
    
    """2^K times 7mod15 control gate"""
    def multi_7mod15_cgate(self, k):
        qc = QuantumCircuit(self.m)
        U = self._7mod15_gate()
        for _ in range(2**k):
            qc.append(U, range(self.m))
    
        U_multi = qc.to_gate()
        U_multi.name = '7Mod15_[2^{}]'.format(k)
    
        cU_multi = U_multi.control()
        return cU_multi
    
    """N-qubit Quantum fourier transformr circuit"""
    @staticmethod
    def qft_qc(n):
        qc = QuantumCircuit(n)
        
        def swap_registers(circuit, n):
            for qubit in range(n//2):
                qc.swap(qubit, n-qubit-1)
            return qc
        
        def qft_rotations(circuit, n):
            """Performs qft on the first n qubits in circuit"""
            if n == 0:
                return circuit
            n -= 1
            qc.h(n)
            for qubit in range(n):
                qc.cp(np.pi/2**(n-qubit), qubit, n)
            qft_rotations(qc, n)

        qft_rotations(qc, n)
        swap_registers(qc, n)
        return qc
    
    def draw(self):
        self.qc.draw()
        
    def get_qc(self):
        return self.qc    

In [8]:
shor = Shor()
shor_qc = shor.get_qc()
shor_qc.draw()

┌───┐                                  ┌───────┐┌─┐      
q_0: ┤ H ├────────■─────────────────────────┤0      ├┤M├──────
     ├───┤        │                         │       │└╥┘┌─┐   
q_1: ┤ H ├────────┼────────────────■────────┤1 QFT+ ├─╫─┤M├───
     ├───┤        │                │        │       │ ║ └╥┘┌─┐
q_2: ┤ H ├────────┼────────────────┼────────┤2      ├─╫──╫─┤M├
     ├───┤┌───────┴───────┐┌───────┴───────┐└───────┘ ║  ║ └╥┘
q_3: ┤ X ├┤0              ├┤0              ├──────────╫──╫──╫─
     └───┘│               ││               │          ║  ║  ║ 
q_4: ─────┤1              ├┤1              ├──────────╫──╫──╫─
          │  7Mod15_[2^0] ││  7Mod15_[2^1] │          ║  ║  ║ 
q_5: ─────┤2              ├┤2              ├──────────╫──╫──╫─
          │               ││               │          ║  ║  ║ 
q_6: ─────┤3              ├┤3              ├──────────╫──╫──╫─
          └───────────────┘└───────────────┘          ║  ║  ║ 
c: 3/═════════════════════════════════════════════════╩══╩══╩═
                                                      0  1  2

In [4]:
sim = Aer.get_backend('aer_simulator')
shots = 20000
shor_trans = transpile(shor_qc, sim)
result = sim.run(shor_trans, shots=shots, memory=True).result() 
count = result.get_counts()
key_new = [str(int(key,2)/2**3) for key in count.keys()]
count_new = dict(zip(key_new, count.values()))
#plot_histogram(count_new)

In [5]:
def amod15():
    sim = Aer.get_backend('aer_simulator')
    shots = 20000
    shor_trans = transpile(shor_qc, sim)
    result = sim.run(shor_trans, shots=shots, memory=True).result() 
    count = result.get_counts()
    key_new = [str(int(key,2)/2**3) for key in count.keys()]
    count_new = dict(zip(key_new, count.values()))
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**shor.m)
    print("Corresponding Phase: %f" % phase)
    return phase

In [6]:
a = 7
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    phase = amod15()
    frac = Fraction(phase).limit_denominator(shor.N)
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        guesses = [gcd(a**(r//2)-1, shor.N), gcd(a**(r//2)+1, shor.N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            if guess not in [1,shor.N] and (shor.N % guess) == 0:
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = True



Attempt 1:
Register Reading: 000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 2:
Register Reading: 100
Corresponding Phase: 0.250000
Result: r = 4
Guessed Factors: 3 and 5
*** Non-trivial factor found: 3 ***
*** Non-trivial factor found: 5 ***


## 6. References

1. Stephane Beauregard, _Circuit for Shor's algorithm using 2n+3 qubits,_ [arXiv:quant-ph/0205095](https://arxiv.org/abs/quant-ph/0205095)

2. [IBM Quantum Experience Shor's algorithm](https://quantum-computing.ibm.com/composer/docs/iqx/guide/shors-algorithm)